# Visualizing annotations

## Set up

In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import numpy as np
import os

import time
from sys import platform

from models import *
from utils.datasets import *
from utils.utils import *
from torch.utils.data import DataLoader
import shutil 
from IPython.display import clear_output
from collections import defaultdict

device = torch_utils.select_device()

Using CUDA device0 _CudaDeviceProperties(name='GeForce GTX 1080 Ti', total_memory=11171MB)


## User input

In [86]:
weights = 'weights/s100e20.pt' # to be filled in 

# default, normally don't change 
cfg = 'cfg/yolov3-spp.cfg'
data_cfg = 'data/GDXray.data'
output = 'output'
img_size = 416
conf_thres = 0.1 # 0.001 in original code 
nms_thres = 0.5 # iou threshold for non-maximum suppression
iou_thres = 0.9 # originally 0.5
batch_size = 32
save_json = False

In [87]:
# Load images that are in testing data in data_cfg 
data_cfg = parse_data_cfg(data_cfg)
nc = int(data_cfg['classes'])  # number of classes
test_path = data_cfg['valid']  # path to test images
names = load_classes(data_cfg['names'])  # class names

In [88]:
# read lines in test file for img names of a few images 
image_ids = []
with open(test_path,"r") as f:
    image_ids += f.readlines()

# Strip all the newlines
image_ids = [p.rstrip() for p in image_ids]

sampled_image_ids = np.random.choice (image_ids, 5)   
print([os.path.basename(p) for p in sampled_image_ids])

['C0001_0069.png', 'C0055_0014.png', 'C0047_0021.png', 'C0007_0019.png', 'C0019_0024.png']


## Plotting help functions

In [89]:
def vivianPlot(img_path, im0, n=0):
    # ground truth 
    plt.subplot(1,2,1)
    image_dir = "data/GDXray/images"
    image_filename = os.path.basename(img_path)
    for roots,_, files in os.walk(image_dir):
        for f in files:
            if f == image_filename:
                impath = os.path.join(roots,f)
                im = np.array(Image.open(impath))
                lpath = impath.replace("images","labels")
                lpath = lpath.replace(".png",".txt")
                break
    # Display the image
    plt.imshow(im)
    
    labels = np.loadtxt(lpath)
    if labels.ndim <= 1: 
        labels = np.array([labels])
    for l in labels:
        w = l[3]*416
        h = l[4]*416
        botx = l[1]*416-w/2
        boty = l[2]*416-h/2
        plt.gca().add_patch(Rectangle((botx,boty),w,h,linewidth=1,edgecolor='r',facecolor='none'))
    
    plt.xlabel(str(len(labels)) + " defects in ground truth")
    
    if n == 0:
        plt.show()
        return 
    
#     predicted 
    plt.subplot(1,2,2)
    plt.imshow(im0)
    plt.xlabel(str(n) + " defects predicted ")
    
    plt.title(os.path.basename(img_path))
    plt.show()

def vivianDetect(detections, img_size, image_path):
    im0 = cv2.imread(image_path) # BGR
    if detections is not None and len(detections) > 0:
        # Rescale boxes from 416 to true image size
        scale_coords(img_size, detections[:, :4], im0.shape).round()

        # Print results to screen
#         for c in detections[:, -1].unique(): # got error because 2 defect boxes overlaped
        for c in range(0):
#             print(detections)
#             print(detections[:,-1])
#             print(c, detections[:, -1].unique())
            n = (detections[:, -1] == c).sum()
            print('%g %ss' % (n, classes[int(c)]), end=', ')

        # Draw bounding boxes and labels of detections
        for *xyxy, conf, cls_conf, cls in detections:
            cls = 0 # added to ignore overlapping 
            color = [0,0,255]

            # Add bbox to the image
            plot_one_box(xyxy, im0, color=color)
#             label = '%s %.2f' % (classes[int(cls)], conf)
#             plot_one_box(xyxy, im0, label=label, color=colors[int(cls)])
    
        n = (detections[:, -1] == 0).sum()
#         vivianPlot(image_path, im0, int(n))
    else: 
        print('0 casting defects', end=', ')
#         vivianPlot(image_path, im0, 0)


## Test

In [90]:
Y = 0 # for counting # defects in truth 
y = 0 # for counting # defects predicted  
num_correct = 0 # number of predicted defects above iou threshold 

torch.set_grad_enabled(False)    
# Initialize model
model = Darknet(cfg, img_size)

# Load weights
if weights.endswith('.pt'):  # pytorch format
    model.load_state_dict(torch.load(weights, map_location=device)['model'])
else:  # darknet format
    _ = load_darknet_weights(model, weights)

model.to(device).eval()

# Dataloader
dataset = LoadImagesAndLabels(test_path, img_size=img_size)
dataloader = DataLoader(dataset,
                        batch_size=batch_size,
                        num_workers=4,
                        pin_memory=False,
                        collate_fn=dataset.collate_fn)

seen = 0
model.eval()
coco91class = coco80_to_coco91_class()
print(('%20s' + '%10s' * 6) % ('Class', 'Images', 'Targets', 'P', 'R', 'mAP', 'F1'))
loss, p, r, f1, mp, mr, map, mf1 = 0., 0., 0., 0., 0., 0., 0., 0.
jdict, stats, ap, ap_class = [], [], [], []
for batch_i, (imgs, targets, paths, shapes) in enumerate(tqdm(dataloader, desc='Computing mAP')):
    targets = targets.to(device)
    imgs = imgs.to(device)

    # Plot images with bounding boxes
    if batch_i == 0 and not os.path.exists('test_batch0.jpg'):
        plot_images(imgs=imgs, targets=targets, fname='test_batch0.jpg')

    # Run model
    inf_out, train_out = model(imgs)  # inference and training outputs

    # Build targets
    target_list = build_targets(model, targets)

    # Compute loss
    loss_i, _ = compute_loss(train_out, target_list)
    loss += loss_i.item()

    # Run NMS
    output = non_max_suppression(inf_out, conf_thres=conf_thres, nms_thres=nms_thres)
    
    # vivianDetect 
    for i in range(len(output)):
        image_path = paths[i]
        if image_path in sampled_image_ids:
            # plot it 
            vivianDetect(output[i],img_size, image_path)
    
    # Statistics per image
    for si, pred in enumerate(output):
        labels = targets[targets[:, 0] == si, 1:]
        nl = len(labels)
        Y += nl
        tcls = labels[:, 0].tolist() if nl else []  # target class
        seen += 1

        if pred is None:
            if nl:
                stats.append(([], torch.Tensor(), torch.Tensor(), tcls))
            continue

        # Assign all predictions as incorrect
        correct = [0] * len(pred)
        y += len(pred)
        if nl:
            detected = []
            tbox = xywh2xyxy(labels[:, 1:5]) * img_size  # target boxes

            # Search for correct predictions
            for i, (*pbox, pconf, pcls_conf, pcls) in enumerate(pred):

                # Break if all targets already located in image
                if len(detected) == nl:
                    break

                # Continue if predicted class not among image classes
                if pcls.item() not in tcls:
                    continue

                # Best iou, index between pred and targets
                iou, bi = bbox_iou(pbox, tbox).max(0)

                # If iou > threshold and class is correct mark as correct
                if iou > iou_thres and bi not in detected:
                    correct[i] = 1
                    num_correct += 1
                    detected.append(bi)
        # Append statistics (correct, conf, pcls, tcls)
        stats.append((correct, pred[:, 4].cpu(), pred[:, 6].cpu(), tcls))

# Compute statistics
stats_np = [np.concatenate(x, 0) for x in list(zip(*stats))]
nt = np.bincount(stats_np[3].astype(np.int64), minlength=nc)  # number of targets per class
if len(stats_np):
    p, r, ap, f1, ap_class = ap_per_class(*stats_np)
    mp, mr, map, mf1 = p.mean(), r.mean(), ap.mean(), f1.mean()

# Print results
pf = '%20s' + '%10.3g' * 6  # print format
print(pf % ('all', seen, nt.sum(), mp, mr, map, mf1), end='\n\n')

# Print results per class
if nc > 1 and len(stats_np):
    for i, c in enumerate(ap_class):
        print(pf % (names[c], seen, nt[c], p[i], r[i], ap[i], f1[i]))

Computing mAP:   0%|          | 0/7 [00:00<?, ?it/s]

               Class    Images   Targets         P         R       mAP        F1


Computing mAP:  71%|███████▏  | 5/7 [00:02<00:00,  2.14it/s]

0 casting defects, 

Computing mAP: 100%|██████████| 7/7 [00:02<00:00,  2.48it/s]

                 all       200       715    0.0162    0.0238   0.00205    0.0193



In [91]:
# # defects in data, # predicted, # predicted correctly
print(Y,y,num_correct)
torch.set_grad_enabled(True)
# plot the results of the same images 


715 1050 17


## Train with new sampling Method (source: medAL paper) 
#### maximize the average distance to all training set examples in a learned feature space (i.e. pixels) 



In [213]:
# input: a list of all trained image paths, a list of all available image paths 
# output: append full image path of the next image that should be annotated to new_metadata
#         a distances_to_trained dictionary storing {img path:np array of distances to all imgs in trained_imgs}
def sampleNextImage (distances_to_trained,
                     trained_image_metadata= "./metadata/GDXray/medAL_sampling.txt", 
                     all_image_metadata = "./metadata/GDXray/castings_shuffled_685.txt", 
                     new_metadata = "./metadata/GDXray/medAL_sampling.txt"):
    if trained_image_metadata != new_metadata:
        shutil.copy(trained_image_metadata, new_metadata)
        
    # parse txt files
    def read_paths(metadata):
        image_paths = []
        with open(metadata,"r") as f:
            image_paths += f.readlines()

        # Strip all the newlines
        image_paths = [p.rstrip() for p in image_paths]
        return image_paths
    trained_image_paths = read_paths(trained_image_metadata)
    all_image_paths = read_paths(all_image_metadata)
    
    # make a list of trained images as np arrays
    trained_images = []
    trained_specimens = defaultdict(int) # {specimen:count of this specimen in trained data}
    for p in trained_image_paths:
        im = cv2.imread(p).flatten() # shape = (416*416*3,)
        trained_images.append(im)
        trained_specimens[os.path.basename(os.path.split(p)[0])] += 1
    
    
    # compute distances 
    untrained_images = []
    max_avg_dist = -1
    max_avg_dist_unique_specimen = -1
    next_image_path = None
    for p in all_image_paths: 
        if p not in trained_image_paths:
            # modify dictionary so that every value is subtracted by min . that way we can pick the specimen with least images
            min_occurance = min(trained_specimens.values())
            for spec in trained_specimens:
                trained_specimens[spec] -= min_occurance
            
            im = cv2.imread(p).flatten()
            untrained_images.append(im)
            if p not in distances_to_trained:  
                distances = np.sum(np.square(im - trained_images), axis = 1) # actually is distance squared, length = # trained img
                distances_to_trained[p] = distances
            else:
                assert(len(distances_to_trained[p]) == len(trained_images)-1)
                distance_to_last_sampled = np.sum(np.square(im - trained_images[-1])) # len = 0
                distances_to_trained[p]=np.append(distances_to_trained[p],distance_to_last_sampled)
                distances = distances_to_trained[p]
            
            # find next best image
            # find the image that has a specimen that has not been chosen yet AND with max avg distance 
            specimen=os.path.basename(os.path.split(p)[0])
            avg_dist = np.mean(distances)
            
            if avg_dist > max_avg_dist_unique_specimen and (specimen not in trained_specimens or trained_specimens[specimen]==0):
                next_image_path = p 
                max_avg_dist_unique_specimen = avg_dist 
            if avg_dist > max_avg_dist:
                max_avg_dist = avg_dist 
                if max_avg_dist_unique_specimen == -1:
                    next_image_path = p
    
    # write next_image_path to new_metadata
    with open(new_metadata,"a") as f: 
        f.write("\n" + next_image_path)
    
    print("Wrote " + next_image_path + " to " + new_metadata)
    print("which now has " + str(len(trained_images) + 1) + " images")

    return distances_to_trained 

In [216]:
import train
import test 

distances_to_trained = {}
training_sizes = [50,100,200,300,400,500,600,685] 
epochs = 200 # start with 200 epochs at 100 training size 

for j in range(len(training_sizes)-1):
    clear_output()
    training_size = training_sizes[j+1]
    inc_size = training_sizes[j+1]-training_sizes[j]
    
    for i in range(inc_size):
        if j ==0 and i == 0: 
            distances_to_trained = sampleNextImage(distances_to_trained, "./metadata/GDXray/castings_shuffled_50.txt")
        else:
            distances_to_trained = sampleNextImage(distances_to_trained)
    
    train.train(cfg = 'cfg/yolov3-spp.cfg',
                data_cfg = 'data/GDXray.data',
                epochs=epochs,
                resume = True)
    shutil.copy('weights/latest.pt','weights/medAL'+str(training_size)+'.pt')
    epochs += 100


In [212]:
dists=sampleNextImage({},
                      "./metadata/GDXray/vivian_sampling.txt",
                      "./metadata/GDXray/castings_shuffled_100.txt",
                      "./metadata/GDXray/vivian_sampling.txt")

Wrote data/GDXray/images/Castings/C0026/C0026_0014.png to ./metadata/GDXray/vivian_sampling.txt
which now has 53 images


In [52]:
dists

{'data/GDXray/images/Castings/C0010/C0010_0030.png': array([39618483, 33565185, 40044417, 40832778, 39675195, 43289613, 42701889, 30460929, 37239843, 39691671, 42055782, 43307739, 37556904], dtype=uint64),
 'data/GDXray/images/Castings/C0001/C0001_0023.png': array([43532907, 46545405, 43404693, 43060260, 45092793, 40723203, 42201585, 40628223, 42528807, 43391025, 20309604, 43166781, 41369772], dtype=uint64)}

In [53]:
for key,value in dists.items():
    print(np.mean(value))

39233879.07692308
41227312.15384615
